# Connecting to an instance of GroundX On-Prem
This notebook assumes you have set up a GroundX On-Prem Instance
https://github.com/eyelevelai/eyelevel-iac

This notebook uses a GroundX On-Prem instance to do parsing and retreival, then passes retreived context to a local LLM, thus doing RAG within a self-contained environment.

Recommended to use an A100 with High-RAM on Google Colab.


In [1]:
!pip3 install groundx

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


Call `kubectl -n eyelevel get svc` to get the external_ip

In [ ]:
#replace with your "external IP"
external_ip = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx-xxxxxxxxxx.us-east-2.elb.amazonaws.com'
api_key = 'xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx'

from groundx import GroundX
client = GroundX(
  api_key=api_key,
  base_url=f"http://{external_ip}/api"
)

# uploading

In [4]:
"""Setting Up Bucket
"""

import json
response = client.buckets.create(
    name="demo_bucket"
)
bucket_id = response.bucket.bucket_id

In [ ]:
"""Uploading Document to Bucket
"""
from groundx import Document

doc_path = 'assets/packageProcess.pdf'

#uploading document
response = client.ingest(
    documents=[
      Document(
        bucket_id=bucket_id,
        file_name=doc_path,
        file_path=doc_path,
        file_type='pdf'
      )
    ]
)

process_id = response.ingest.process_id
process_id

'b9ff24c7-61d8-420b-8337-c6b70f69371d'

In [6]:
"""Checking Status
wait for it to "status":"complete" before progressing
"""
import time
while True:

    response = client.documents.get_processing_status_by_id(
        process_id=process_id
    )

    if response.ingest.status == 'complete':
        print('done!')
        break

    print('still processing...')
    time.sleep(10)

still processing...
still processing...
still processing...
still processing...
still processing...
still processing...
still processing...
still processing...
still processing...
still processing...
still processing...
still processing...
still processing...
still processing...
done!


# Retreival

In [8]:
import json

query = "What testing protocol should I follow if the package is shipping LTL?"

response = client.search.content(
    id=bucket_id,
    query=query
)
context = response.search.text

In [ ]:
print(context)

# Making an Inference with an LLM Locally

In [ ]:
import os
from huggingface_hub import login
from google.colab import userdata

login(userdata.get('HuggingFace'))

In [ ]:
from transformers import pipeline
pipe = pipeline("text-generation", model="meta-llama/Llama-3.2-3B-Instruct", max_length = 6000, device='cuda')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

messages = [
    {"role": "user", "content": f"use this information to answer my next query\n\n{context}"},
    {"role": "user", "content": f'{query} Think about your answer then generate a response'},
]
print(pipe(messages)[0]['generated_text'][-1]['content'])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Based on the provided text excerpts, if the package is shipping LTL (Less Than Truckload), the testing protocol to follow is the ISTA 3B protocol. This is because the flowchart indicates that if the item ships LTL from the Fulfillment Center, the user should follow the ISTA 3B protocol.

Additionally, the text excerpt from page 3 states that if the item does not meet the conditions for shipping SIOC, it will be shipped via LTL. Therefore, if the package is shipping LTL, it is likely that the ISTA 3B protocol should be followed.

It's worth noting that the text excerpt from page 2 also mentions that if any of the conditions are met, the item will ship LTL. These conditions include:

* >150lbs
* >108" in longest dimension (9ft)
* >165" in length (L) + girth (2W+2H)

If any of these conditions are met, the package will ship LTL and the ISTA 3B protocol should be followed.
